In [1]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook
import glob
import os
pd.options.display.float_format = '{:,.2f}'.format
import json
import csv

### PARAMETRIA COMISIONES

In [2]:
parametria_comisiones = pd.read_csv(r'G:\My Drive\RESPALDO-PC\TOPUPS\PARAMETRIA TOPUPS.csv',dtype={'monto_recarga':float})
parametria_comisiones['inicio'] = pd.to_datetime(parametria_comisiones['inicio'])
parametria_comisiones['porcentaje_comision_final'] = parametria_comisiones['porcentaje_comision']/100
parametria_comisiones['index_comisiones'] = (parametria_comisiones['carrier'].astype(str).replace(' ', '',regex=True)+'_'
                                             +parametria_comisiones['monto_recarga'].astype(str))
parametria_comisiones.head()

,motivo_del_cambio,naturaleza,estatus,inicio,fin,carrier,monto_recarga,porcentaje_comision,categoria,porcentaje_comision_final,index_comisiones
0,NO APLICA,FIJA,VIGENTE,2020-01-08,2999-12-31,MOVISTAR,30.00,7.00,MOVISTAR,0.07,MOVISTAR_30.0
1,NO APLICA,FIJA,VIGENTE,2020-01-08,2999-12-31,MOVISTAR,50.00,7.00,MOVISTAR,0.07,MOVISTAR_50.0
2,NO APLICA,FIJA,VIGENTE,2020-01-08,2999-12-31,MOVISTAR,100.00,7.00,MOVISTAR,0.07,MOVISTAR_100.0
3,NO APLICA,FIJA,VIGENTE,2020-01-08,2999-12-31,MOVISTAR,200.00,7.00,MOVISTAR,0.07,MOVISTAR_200.0
4,NO APLICA,FIJA,VIGENTE,2020-01-08,2999-12-31,MOVISTAR,500.00,7.00,MOVISTAR,0.07,MOVISTAR_500.0


In [3]:
parametria_comisiones.shape

(26, 11)

In [4]:
parametria_comisiones.dtypes

motivo_del_cambio                    object
naturaleza                           object
estatus                              object
inicio                       datetime64[ns]
fin                                  object
carrier                              object
monto_recarga                       float64
porcentaje_comision                 float64
categoria                            object
porcentaje_comision_final           float64
index_comisiones                     object
dtype: object

In [5]:
parametria_comisiones['carrier'].unique()

array(['MOVISTAR', 'UNEFON', 'TELCEL', 'ATT', 'IUSACELL'], dtype=object)

In [6]:
parametria_comisiones['index_comisiones'].unique()

array(['MOVISTAR_30.0', 'MOVISTAR_50.0', 'MOVISTAR_100.0',
       'MOVISTAR_200.0', 'MOVISTAR_500.0', 'UNEFON_30.0', 'UNEFON_50.0',
       'UNEFON_100.0', 'UNEFON_200.0', 'UNEFON_500.0', 'TELCEL_30.0',
       'TELCEL_50.0', 'TELCEL_80.0', 'TELCEL_100.0', 'TELCEL_200.0',
       'TELCEL_500.0', 'ATT_30.0', 'ATT_50.0', 'ATT_100.0', 'ATT_200.0',
       'ATT_500.0', 'IUSACELL_30.0', 'IUSACELL_50.0', 'IUSACELL_100.0',
       'IUSACELL_200.0', 'IUSACELL_500.0'], dtype=object)

### SERTI (EXTRAIDO EN EXCEL)

In [7]:
serti_files = glob.glob(r'G:\My Drive\RESPALDO-PC\TOPUPS\202*\**\SERTI\*.csv',recursive=False)
serti_files

['G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\05. MAYO\\SERTI\\result.csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\05. MAYO\\SERTI\\result (1).csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\05. MAYO\\SERTI\\result (2).csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\05. MAYO\\SERTI\\result (3).csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\05. MAYO\\SERTI\\result (4).csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\05. MAYO\\SERTI\\result (5).csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\05. MAYO\\SERTI\\result (6).csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\05. MAYO\\SERTI\\result (7).csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\05. MAYO\\SERTI\\result (8).csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\05. MAYO\\SERTI\\result (9).csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\05. MAYO\\SERTI\\result (10).csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\05. MAYO\\SERTI\\result (11).csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\05. MAYO\\SERTI\\result (12).csv',


In [10]:
serti = pd.concat([pd.read_csv(f,dtype={'bin':object,'panTerminacion':object,'claveRechazo':object,'phone':object
                                        ,'transactionId':object,'importe':float})
                   .assign(file_name=os.path.basename(f)) for f in serti_files])
#serti.loc[serti['carrier'].str.contains('AT&T'), 'carrier'] = 'ATT'
serti['carrier'] = serti['carrier'].str.strip()
serti['carrier'] = serti['carrier'].str.upper()
serti['last4_phone_serti'] = serti['phone'].str[-4:]
serti['serti_date'] = serti['date'].str[:10]
serti['serti_date'] = pd.to_datetime(serti['serti_date'])
serti.sort_values(by=['date'],inplace=True)
serti['INDEX_TOTAL'] = (serti['transactionId'].astype(str).replace(' ', '',regex=True) + '_'
                       + serti['last4_phone_serti'].astype(str).replace(' ', '',regex=True) + '_'
                       + serti['panTerminacion'].astype(str).replace(' ', '',regex=True) + '_'
                       + serti['importe'].astype(str).replace(' ', '',regex=True))
serti['index_comisiones'] = (serti['carrier'].astype(str).replace(' ', '',regex=True) + '_'
                       + serti['importe'].astype(str).replace(' ', '',regex=True))
serti.reset_index(inplace=True)
serti.head()

,index,bin,panTerminacion,importe,date,transactionId,entryMode,numAuth,terminal,caja,...,ivaComision,carrier,phone,folio,file_name,claveVenta,last4_phone_serti,serti_date,INDEX_TOTAL,index_comisiones
0,7954,402766,7364,50.00,2021-05-01T00:00:21.918,34205977,Chip,"482,732.00",nan,0,...,0,UNEFON,*******4162,"502,474.00",result.csv,nan,4162,2021-05-01,34205977_4162_7364_50.0,UNEFON_50.0
1,7953,415231,5366,100.00,2021-05-01T00:01:24.31,34206011,Chip,"318,315.00",nan,0,...,0,TELCEL,*******0854,"458,139.00",result.csv,nan,0854,2021-05-01,34206011_0854_5366_100.0,TELCEL_100.0
2,7952,415231,7946,100.00,2021-05-01T00:02:20.669,34206035,Chip,"321,173.00",nan,0,...,0,TELCEL,*******8827,"143,713.00",result.csv,nan,8827,2021-05-01,34206035_8827_7946_100.0,TELCEL_100.0
3,7951,412408,2697,50.00,2021-05-01T00:03:08.02,34206062,Chip,"649,428.00",nan,0,...,0,TELCEL,*******7918,"19,708.00",result.csv,nan,7918,2021-05-01,34206062_7918_2697_50.0,TELCEL_50.0
4,7950,404313,3500,50.00,2021-05-01T00:04:12.187,34206091,Chip,"863,609.00",nan,0,...,0,IUSACELL,*******1258,"503,435.00",result.csv,nan,1258,2021-05-01,34206091_1258_3500_50.0,IUSACELL_50.0


In [11]:
serti.shape

(2106455, 26)

In [12]:
serti.columns

Index(['index', 'bin', 'panTerminacion', 'importe', 'date', 'transactionId',
       'entryMode', 'numAuth', 'terminal', 'caja', 'aprobada', 'claveRechazo',
       'merchant', 'producto', 'tarjetahabiente', 'comision', 'ivaComision',
       'carrier', 'phone', 'folio', 'file_name', 'claveVenta',
       'last4_phone_serti', 'serti_date', 'INDEX_TOTAL', 'index_comisiones'],
      dtype='object')

In [13]:
serti['index_comisiones'].unique()

array(['UNEFON_50.0', 'TELCEL_100.0', 'TELCEL_50.0', 'IUSACELL_50.0',
       'UNEFON_30.0', 'MOVISTAR_30.0', 'MOVISTAR_50.0', 'UNEFON_100.0',
       'IUSACELL_100.0', 'TELCEL_30.0', 'IUSACELL_30.0', 'IUSACELL_200.0',
       'MOVISTAR_100.0', 'TELCEL_200.0', 'TELCEL_80.0', 'MOVISTAR_500.0',
       'MOVISTAR_200.0', 'IUSACELL_500.0', 'TELCEL_500.0', 'UNEFON_200.0',
       'UNEFON_500.0', 'UNEFON_3000.0', 'nan_50.0', 'nan_100.0',
       'AT&T_30.0', 'AT&T_100.0', 'AT&T_50.0', 'nan_30.0', 'nan_200.0',
       'AT&T_200.0', 'nan_80.0', 'nan_500.0', 'AT&T_500.0'], dtype=object)

In [14]:
duplicates_serti = serti[serti.duplicated(['transactionId'] , keep=False)]
duplicates_serti.head()

,index,bin,panTerminacion,importe,date,transactionId,entryMode,numAuth,terminal,caja,...,ivaComision,carrier,phone,folio,file_name,claveVenta,last4_phone_serti,serti_date,INDEX_TOTAL,index_comisiones
353733,6405,421316,1149,100.00,2021-06-20T00:00:13.774,37812948,Chip,"57,708.00",nan,0,...,0,UNEFON,9842888311,804866,result (19).csv,nan,8311,2021-06-20,37812948_8311_1149_100.0,UNEFON_100.0
353734,6405,421316,1149,100.00,2021-06-20T00:00:13.774,37812948,Chip,"57,708.00",nan,0,...,0,UNEFON,9842888311,804866,result (18).csv,nan,8311,2021-06-20,37812948_8311_1149_100.0,UNEFON_100.0
353735,6409,415231,7199,50.00,2021-06-20T00:00:27.099,37812912,Chip,"220,282.00",nan,0,...,0,UNEFON,8993335289,804835,result (18).csv,nan,5289,2021-06-20,37812912_5289_7199_50.0,UNEFON_50.0
353736,6408,415231,7199,50.00,2021-06-20T00:00:27.099,37812912,Chip,"220,282.00",nan,0,...,0,UNEFON,8993335289,804835,result (19).csv,nan,5289,2021-06-20,37812912_5289_7199_50.0,UNEFON_50.0
353737,6404,520416,0590,30.00,2021-06-20T00:00:31.474,37812952,Chip,"19,801.00",nan,0,...,0,TELCEL,8445332124,731780,result (18).csv,nan,2124,2021-06-20,37812952_2124_0590_30.0,TELCEL_30.0


In [15]:
duplicates_serti.shape

(12818, 26)

In [16]:
serti.drop_duplicates(subset='INDEX_TOTAL', keep='first', inplace=True)

In [17]:
serti.shape

(2100046, 26)

In [18]:
serti.dtypes

index                         int64
bin                          object
panTerminacion               object
importe                     float64
date                         object
transactionId                object
entryMode                    object
numAuth                      object
terminal                    float64
caja                          int64
aprobada                       bool
claveRechazo                 object
merchant                      int64
producto                     object
tarjetahabiente              object
comision                      int64
ivaComision                   int64
carrier                      object
phone                        object
folio                        object
file_name                    object
claveVenta                  float64
last4_phone_serti            object
serti_date           datetime64[ns]
INDEX_TOTAL                  object
index_comisiones             object
dtype: object

### SERTI CON PARAMETRÍA DE COMISIONES

In [19]:
serti_final = serti.merge(parametria_comisiones[['index_comisiones','estatus','porcentaje_comision_final']]
                      ,on='index_comisiones',how='left')
serti_final['estatus'].fillna('Not_found',inplace=True)
serti_final['porcentaje_comision_final'].fillna(0,inplace=True)
serti_final['comision_db'] = serti_final['importe'] * serti_final['porcentaje_comision_final']
serti_final['iva_comision_db'] = serti_final['comision_db'] * 0.1600
serti_final.head()

,index,bin,panTerminacion,importe,date,transactionId,entryMode,numAuth,terminal,caja,...,file_name,claveVenta,last4_phone_serti,serti_date,INDEX_TOTAL,index_comisiones,estatus,porcentaje_comision_final,comision_db,iva_comision_db
0,7954,402766,7364,50.00,2021-05-01T00:00:21.918,34205977,Chip,"482,732.00",nan,0,...,result.csv,nan,4162,2021-05-01,34205977_4162_7364_50.0,UNEFON_50.0,VIGENTE,0.07,3.50,0.56
1,7953,415231,5366,100.00,2021-05-01T00:01:24.31,34206011,Chip,"318,315.00",nan,0,...,result.csv,nan,0854,2021-05-01,34206011_0854_5366_100.0,TELCEL_100.0,VIGENTE,0.04,4.00,0.64
2,7952,415231,7946,100.00,2021-05-01T00:02:20.669,34206035,Chip,"321,173.00",nan,0,...,result.csv,nan,8827,2021-05-01,34206035_8827_7946_100.0,TELCEL_100.0,VIGENTE,0.04,4.00,0.64
3,7951,412408,2697,50.00,2021-05-01T00:03:08.02,34206062,Chip,"649,428.00",nan,0,...,result.csv,nan,7918,2021-05-01,34206062_7918_2697_50.0,TELCEL_50.0,VIGENTE,0.04,2.00,0.32
4,7950,404313,3500,50.00,2021-05-01T00:04:12.187,34206091,Chip,"863,609.00",nan,0,...,result.csv,nan,1258,2021-05-01,34206091_1258_3500_50.0,IUSACELL_50.0,VIGENTE,0.07,3.50,0.56


In [20]:
serti_final.shape

(2100046, 30)

### PIVOT SERTI

In [21]:
pivot_serti = pd.pivot_table(serti_final,index=['serti_date']
                            ,columns=['aprobada']
                            ,values=['importe','transactionId','comision_db','iva_comision_db']
                            ,aggfunc={'importe':np.sum,
                                     'transactionId':len,
                                     'comision_db':np.sum,
                                     'iva_comision_db':np.sum,},margins=True).reset_index()
pivot_serti.fillna(0,inplace=True)
pivot_serti.head()

serti_date comision_db                       importe  \
aprobada                            False      True       All     False   
0         2021-05-01 00:00:00    2,985.20 23,633.40 26,618.60 57,680.00   
1         2021-05-02 00:00:00    1,701.00 17,828.10 19,529.10 33,630.00   
2         2021-05-03 00:00:00    2,290.40 23,834.40 26,124.80 43,850.00   
3         2021-05-04 00:00:00    2,287.20 20,783.30 23,070.50 43,830.00   
4         2021-05-05 00:00:00    2,046.30 20,257.70 22,304.00 37,260.00   

                               iva_comision_db                    \
aprobada       True        All           False     True      All   
0        461,550.00 519,230.00          477.63 3,781.34 4,258.98   
1        349,680.00 383,310.00          272.16 2,852.50 3,124.66   
2        465,720.00 509,570.00          366.46 3,813.50 4,179.97   
3        409,100.00 452,930.00          365.95 3,325.33 3,691.28   
4        396,050.00 433,310.00          327.41 3,241.23 3,568.64   

         transactionId                 
aprobada         False     True   All  
0               857.00 7,098.00  7955  
1               530.00 5,429.00  5959  
2               692.00 7,142.00  7834  
3               675.00 6,268.00  6943  
4               553.00 6,080.00  6633

In [22]:
pivot_serti.shape

(267, 13)

### DB PRODUCTO

In [23]:
name_file = ['merchant_id','topup_id','transaction_id','clip_external_tracking_id','carrier','phone_number','created_at',
             'updated_at','created_at_datetime','updated_at_datetime','updated_at_og','payment_type','status_code',
             'status_msg','amount','processor','issuer_auth_code','folio_id','receipt_no','last4','authNum','orderId',
             'serti_transaction_id','type','response_code']

In [24]:
topups_db = pd.read_csv(r'C:\Users\Cristian.Aparicio\Documents\TOPUPS\REVIEW\2022\01. ENERO\DATA\REVIEW TOPUPS 2022.01.27.txt',
                              sep='\t',names=name_file,dtype={'clip_external_tracking_id':object,'phone_number':object,'last4':object,'amount':float,
                                     'status_code':object,'serti_transaction_id':object})
topups_db['clip_external_tracking_id'].fillna('sin_order_id_db',inplace=True)
topups_db['serti_transaction_id'].fillna('sin_order_id_db',inplace=True)
topups_db['created_at'] = pd.to_datetime(topups_db['created_at'])
topups_db['updated_at'] = pd.to_datetime(topups_db['updated_at'])
topups_db['last4_phone'] = topups_db['phone_number'].str[-4:]
topups_db['INDEX_TOTAL'] = (topups_db['serti_transaction_id'].astype(str).replace(' ', '',regex=True) + '_'
                           + topups_db['last4_phone'].astype(str).replace(' ', '',regex=True) + '_'
                           + topups_db['last4'].astype(str).replace(' ', '',regex=True) + '_'
                           + topups_db['amount'].astype(str).replace(' ', '',regex=True))
topups_db['index_comisiones'] = (topups_db['carrier'].astype(str).replace(' ', '',regex=True)+'-'
                                             +topups_db['amount'].astype(str))
topups_db.head()

,merchant_id,topup_id,transaction_id,clip_external_tracking_id,carrier,phone_number,created_at,updated_at,created_at_datetime,updated_at_datetime,...,receipt_no,last4,authNum,orderId,serti_transaction_id,type,response_code,last4_phone,INDEX_TOTAL,index_comisiones
0,da4ed924-5091-4aeb-a876-fd8c64ab76f5,5a39bc48-fb09-44a4-adcb-eadf2f13bcbe,bf6e98fe-e457-4a4d-9cc1-6f1f7f44a1ed,sin_order_id_db,ATT,6461215663,2021-10-01,2021-10-01,2021-10-01 00:01:28,2021-10-01 00:01:28,...,TU-Z2i4Zrp,1893,nan,41204798,sin_order_id_db,emv_sale,H009,5663,sin_order_id_db_5663_1893_200.0,ATT-200.0
1,3ee649c0-e6d9-4492-ad0c-482e15b6fc70,1210a460-eab7-4322-af15-762c6c7efb13,15f402d9-1eae-47ea-884b-239c816896e4,sin_order_id_db,TELCEL,7713236589,2021-10-01,2021-10-01,2021-10-01 00:01:51,2021-10-01 00:01:51,...,TU-tSShuS5,7572,nan,41204806,sin_order_id_db,emv_sale,62,6589,sin_order_id_db_6589_7572_30.0,TELCEL-30.0
2,0952d88e-c307-4371-98ae-7de6b8398584,2b1e3abc-4131-4702-9aec-cfbcd0e1017b,17a7703e-feef-451f-9635-8130113389e5,sin_order_id_db,ATT,3323859948,2021-10-01,2021-10-01,2021-10-01 00:05:43,2021-10-01 00:05:43,...,TU-Dpnea9X,4369,nan,41204855,sin_order_id_db,emv_sale,68,9948,sin_order_id_db_9948_4369_100.0,ATT-100.0
3,b5ab702b-7481-49ce-8772-5d2b5a000af7,302c6ca8-6d65-4ad7-9b73-8c09f60f1cc4,df3a1373-4ad8-47d2-93d1-bdd022bd8bd1,sin_order_id_db,MOVISTAR,9982634649,2021-10-01,2021-10-01,2021-10-01 00:07:28,2021-10-01 00:07:28,...,TU-Z7PJgd9,5193,nan,41204871,sin_order_id_db,emv_sale,51,4649,sin_order_id_db_4649_5193_100.0,MOVISTAR-100.0
4,9505e71d-a955-49c3-8a12-1cca80dee256,5cdc8284-dbd5-47f1-b7a7-66e8f54cdbac,992ab446-d534-479d-a702-b268bfbf6255,sin_order_id_db,TELCEL,6311958885,2021-10-01,2021-10-01,2021-10-01 00:26:15,2021-10-01 00:26:15,...,TU-aAdFvuS,1820,nan,41205040,sin_order_id_db,emv_sale,5,8885,sin_order_id_db_8885_1820_50.0,TELCEL-50.0


In [25]:
topups_db.shape

(91780, 28)

In [26]:
topups_db['carrier'].unique()

array(['ATT', 'TELCEL', 'MOVISTAR', 'UNEFON'], dtype=object)

In [27]:
duplicates = topups_db[topups_db.duplicated(['serti_transaction_id'], keep=False)]
duplicates.head()

,merchant_id,topup_id,transaction_id,clip_external_tracking_id,carrier,phone_number,created_at,updated_at,created_at_datetime,updated_at_datetime,...,receipt_no,last4,authNum,orderId,serti_transaction_id,type,response_code,last4_phone,INDEX_TOTAL,index_comisiones
0,da4ed924-5091-4aeb-a876-fd8c64ab76f5,5a39bc48-fb09-44a4-adcb-eadf2f13bcbe,bf6e98fe-e457-4a4d-9cc1-6f1f7f44a1ed,sin_order_id_db,ATT,6461215663,2021-10-01,2021-10-01,2021-10-01 00:01:28,2021-10-01 00:01:28,...,TU-Z2i4Zrp,1893,nan,41204798,sin_order_id_db,emv_sale,H009,5663,sin_order_id_db_5663_1893_200.0,ATT-200.0
1,3ee649c0-e6d9-4492-ad0c-482e15b6fc70,1210a460-eab7-4322-af15-762c6c7efb13,15f402d9-1eae-47ea-884b-239c816896e4,sin_order_id_db,TELCEL,7713236589,2021-10-01,2021-10-01,2021-10-01 00:01:51,2021-10-01 00:01:51,...,TU-tSShuS5,7572,nan,41204806,sin_order_id_db,emv_sale,62,6589,sin_order_id_db_6589_7572_30.0,TELCEL-30.0
2,0952d88e-c307-4371-98ae-7de6b8398584,2b1e3abc-4131-4702-9aec-cfbcd0e1017b,17a7703e-feef-451f-9635-8130113389e5,sin_order_id_db,ATT,3323859948,2021-10-01,2021-10-01,2021-10-01 00:05:43,2021-10-01 00:05:43,...,TU-Dpnea9X,4369,nan,41204855,sin_order_id_db,emv_sale,68,9948,sin_order_id_db_9948_4369_100.0,ATT-100.0
3,b5ab702b-7481-49ce-8772-5d2b5a000af7,302c6ca8-6d65-4ad7-9b73-8c09f60f1cc4,df3a1373-4ad8-47d2-93d1-bdd022bd8bd1,sin_order_id_db,MOVISTAR,9982634649,2021-10-01,2021-10-01,2021-10-01 00:07:28,2021-10-01 00:07:28,...,TU-Z7PJgd9,5193,nan,41204871,sin_order_id_db,emv_sale,51,4649,sin_order_id_db_4649_5193_100.0,MOVISTAR-100.0
4,9505e71d-a955-49c3-8a12-1cca80dee256,5cdc8284-dbd5-47f1-b7a7-66e8f54cdbac,992ab446-d534-479d-a702-b268bfbf6255,sin_order_id_db,TELCEL,6311958885,2021-10-01,2021-10-01,2021-10-01 00:26:15,2021-10-01 00:26:15,...,TU-aAdFvuS,1820,nan,41205040,sin_order_id_db,emv_sale,5,8885,sin_order_id_db_8885_1820_50.0,TELCEL-50.0


In [28]:
duplicates.shape

(17175, 28)

In [29]:
duplicates['serti_transaction_id'].unique()

array(['sin_order_id_db', '0'], dtype=object)

In [30]:
topups_db.shape

(91780, 28)

In [31]:
topups_db.dtypes

merchant_id                          object
topup_id                             object
transaction_id                       object
clip_external_tracking_id            object
carrier                              object
phone_number                         object
created_at                   datetime64[ns]
updated_at                   datetime64[ns]
created_at_datetime                  object
updated_at_datetime                  object
updated_at_og                        object
payment_type                         object
status_code                          object
status_msg                           object
amount                              float64
processor                            object
issuer_auth_code                    float64
folio_id                            float64
receipt_no                           object
last4                                object
authNum                             float64
orderId                               int64
serti_transaction_id            

In [32]:
# topups_db.to_excel('DB_PRODUCT.xlsx')

### DB CON PARAMETRÍA DE COMISIONES

In [33]:
topups_db_final = topups_db

In [34]:
# topups_db_final = topups_db.merge(parametria_comisiones[['index_comisiones','estatus','porcentaje_comision_final']]
#                       ,on='index_comisiones',how='left')
# topups_db_final['estatus'].fillna('Not_found',inplace=True)
# topups_db_final['porcentaje_comision_final'].fillna(0,inplace=True)
# topups_db_final['comision_db'] = topups_db_final['amount'] * topups_db_final['porcentaje_comision_final']
# topups_db_final['iva_comision_db'] = topups_db_final['comision_db'] * 0.1600
# topups_db_final.head()

In [35]:
topups_db_final.shape

(91780, 28)

### SERTI VS DB

In [36]:
cruce_1 = serti_final.merge(topups_db_final[['INDEX_TOTAL','topup_id','transaction_id','created_at','updated_at','status_code','status_msg'
                                 ,'amount','last4_phone','serti_transaction_id','carrier','type','response_code']]
                      ,on='INDEX_TOTAL',how='inner')
cruce_1['status_code'].fillna('Not_found',inplace=True)
cruce_1['topup_id'].fillna('Not_found',inplace=True)
cruce_1.head()

,index,bin,panTerminacion,importe,date,transactionId,entryMode,numAuth,terminal,caja,...,created_at,updated_at,status_code,status_msg,amount,last4_phone,serti_transaction_id,carrier_y,type,response_code
0,647,557909,2538,50.00,2021-10-07T21:25:17.157,45292539,Chip,NaN,nan,0,...,2021-10-07,2021-10-07,-3,Declined by issuer,50.00,1723,45292539,ATT,emv_sale,12
1,633,415231,2100,100.00,2021-10-07T21:27:32.151,45292660,Chip,NaN,nan,0,...,2021-10-07,2021-10-07,-3,Declined by issuer,100.00,7028,45292660,TELCEL,emv_sale,51
2,613,511114,8459,50.00,2021-10-07T21:31:02.413,45292850,Chip,NaN,nan,0,...,2021-10-07,2021-10-07,-3,Declined by issuer,50.00,1470,45292850,TELCEL,emv_sale,51
3,550,416916,3621,100.00,2021-10-07T21:40:02.611,45293339,Chip,NaN,nan,0,...,2021-10-07,2021-10-07,-3,Declined by issuer,100.00,5039,45293339,TELCEL,emv_sale,51
4,532,557907,8649,100.00,2021-10-07T21:41:51.619,45293444,Chip,NaN,nan,0,...,2021-10-07,2021-10-07,-3,Declined by issuer,100.00,7031,45293444,TELCEL,emv_sale,51


In [37]:
cruce_1.shape

(74605, 42)

### TRUE ONLY

In [38]:
cruce_1_true = cruce_1[(cruce_1['aprobada'] == True)]
cruce_1_true['amount'].fillna(0,inplace=True)
cruce_1_true.head()

C:\Users\Cristian.Aparicio\anaconda3\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


,index,bin,panTerminacion,importe,date,transactionId,entryMode,numAuth,terminal,caja,...,created_at,updated_at,status_code,status_msg,amount,last4_phone,serti_transaction_id,carrier_y,type,response_code
58727,5262,547046,0851,100.00,2021-12-22T13:06:36.049,49350494,Chip,"63,542.00",nan,0,...,2021-12-22,2021-12-22,-5,Internal Error,100.00,1417,49350494,TELCEL,emv_sale,00
58728,5260,525678,1497,50.00,2021-12-22T13:06:36.278,49350496,Chip,"6,867.00",nan,0,...,2021-12-22,2021-12-22,-5,Internal Error,50.00,2786,49350496,TELCEL,emv_sale,00
73028,2440,557920,1319,100.00,2022-01-11T17:22:12.091,50419584,Chip,"550,830.00",nan,0,...,2022-01-11,2022-01-11,0,Pending,100.00,3140,50419584,TELCEL,emv_sale,00
73029,2438,526354,7150,100.00,2022-01-11T17:22:13.832,50419588,Chip,"115,130.00",nan,0,...,2022-01-11,2022-01-11,0,Pending,100.00,0392,50419588,ATT,emv_sale,00


In [39]:
cruce_1_true.shape

(4, 42)

In [40]:
duplicates_cruce_1_true = cruce_1_true[cruce_1_true.duplicated(['INDEX_TOTAL'] , keep=False)]
duplicates_cruce_1_true.head()

,index,bin,panTerminacion,importe,date,transactionId,entryMode,numAuth,terminal,caja,...,created_at,updated_at,status_code,status_msg,amount,last4_phone,serti_transaction_id,carrier_y,type,response_code


In [41]:
duplicates_cruce_1_true.shape

(0, 42)

In [42]:
cruce_1_final = cruce_1_true[(cruce_1_true['topup_id'] != 'Not_found')]
cruce_1_final.head()

,index,bin,panTerminacion,importe,date,transactionId,entryMode,numAuth,terminal,caja,...,created_at,updated_at,status_code,status_msg,amount,last4_phone,serti_transaction_id,carrier_y,type,response_code
58727,5262,547046,0851,100.00,2021-12-22T13:06:36.049,49350494,Chip,"63,542.00",nan,0,...,2021-12-22,2021-12-22,-5,Internal Error,100.00,1417,49350494,TELCEL,emv_sale,00
58728,5260,525678,1497,50.00,2021-12-22T13:06:36.278,49350496,Chip,"6,867.00",nan,0,...,2021-12-22,2021-12-22,-5,Internal Error,50.00,2786,49350496,TELCEL,emv_sale,00
73028,2440,557920,1319,100.00,2022-01-11T17:22:12.091,50419584,Chip,"550,830.00",nan,0,...,2022-01-11,2022-01-11,0,Pending,100.00,3140,50419584,TELCEL,emv_sale,00
73029,2438,526354,7150,100.00,2022-01-11T17:22:13.832,50419588,Chip,"115,130.00",nan,0,...,2022-01-11,2022-01-11,0,Pending,100.00,0392,50419588,ATT,emv_sale,00


In [43]:
cruce_1_final.shape

(4, 42)

In [44]:
na_cruce_1_final = cruce_1_true[(cruce_1_true['topup_id'] == 'Not_found')]
na_cruce_1_final.head()

,index,bin,panTerminacion,importe,date,transactionId,entryMode,numAuth,terminal,caja,...,created_at,updated_at,status_code,status_msg,amount,last4_phone,serti_transaction_id,carrier_y,type,response_code


In [45]:
na_cruce_1_final.shape

(0, 42)

In [46]:
cruce_1_true.columns

Index(['index', 'bin', 'panTerminacion', 'importe', 'date', 'transactionId',
       'entryMode', 'numAuth', 'terminal', 'caja', 'aprobada', 'claveRechazo',
       'merchant', 'producto', 'tarjetahabiente', 'comision', 'ivaComision',
       'carrier_x', 'phone', 'folio', 'file_name', 'claveVenta',
       'last4_phone_serti', 'serti_date', 'INDEX_TOTAL', 'index_comisiones',
       'estatus', 'porcentaje_comision_final', 'comision_db',
       'iva_comision_db', 'topup_id', 'transaction_id', 'created_at',
       'updated_at', 'status_code', 'status_msg', 'amount', 'last4_phone',
       'serti_transaction_id', 'carrier_y', 'type', 'response_code'],
      dtype='object')

In [47]:
cruce_1.dtypes

index                                 int64
bin                                  object
panTerminacion                       object
importe                             float64
date                                 object
transactionId                        object
entryMode                            object
numAuth                              object
terminal                            float64
caja                                  int64
aprobada                               bool
claveRechazo                         object
merchant                              int64
producto                             object
tarjetahabiente                      object
comision                              int64
ivaComision                           int64
carrier_x                            object
phone                                object
folio                                object
file_name                            object
claveVenta                          float64
last4_phone_serti               

### PIVOT TRUE ONLY

In [48]:
pivot_true = pd.pivot_table(cruce_1_true,index=['serti_date']
                            ,columns=['status_code']
                            ,values=['importe','amount','transactionId','serti_transaction_id','comision_db',
                                     'iva_comision_db']
                            ,aggfunc={'importe':np.sum,
                                      'amount':np.sum,
                                      'comision_db':np.sum,
                                      'iva_comision_db':np.sum,
                                      'transactionId':len,
                                      'serti_transaction_id':len},margins=True).reset_index()
pivot_true.fillna(0,inplace=True)
pivot_true.head()

serti_date amount               comision_db             \
status_code                          -5      0    All          -5    0   All   
0            2021-12-22 00:00:00 150.00   0.00 150.00        6.00 0.00  6.00   
1            2022-01-11 00:00:00   0.00 200.00 200.00        0.00 4.00  4.00   
2                            All 150.00 200.00 350.00        6.00 4.00 10.00   

            importe               iva_comision_db            \
status_code      -5      0    All              -5    0  All   
0            150.00   0.00 150.00            0.96 0.00 0.96   
1              0.00 200.00 200.00            0.00 0.64 0.64   
2            150.00 200.00 350.00            0.96 0.64 1.60   

            serti_transaction_id          transactionId           
status_code                   -5    0 All            -5    0 All  
0                           2.00 0.00   2          2.00 0.00   2  
1                           0.00 2.00   2          0.00 2.00   2  
2                           2.00 2.00   4          2.00 2.00   4

In [49]:
pivot_true.shape

(3, 19)

### PIVOT TRUE Y CARRIER 

In [50]:
pivot_carrier = pd.pivot_table(cruce_1_true,index=['serti_date','carrier_x']
                            ,columns=['status_code']
                            ,values=['importe','amount','transactionId','serti_transaction_id','comision_db',
                                     'iva_comision_db']
                            ,aggfunc={'importe':np.sum,
                                      'amount':np.sum,
                                      'comision_db':np.sum,
                                      'iva_comision_db':np.sum,
                                      'transactionId':len,
                                      'serti_transaction_id':len},margins=True).reset_index()
pivot_carrier.fillna(0,inplace=True)
pivot_carrier.head()

serti_date carrier_x amount               comision_db  \
status_code                                    -5      0    All          -5   
0            2021-12-22 00:00:00    TELCEL 150.00   0.00 150.00        6.00   
1            2022-01-11 00:00:00      AT&T   0.00 100.00 100.00        0.00   
2            2022-01-11 00:00:00    TELCEL   0.00 100.00 100.00        0.00   
3                            All           150.00 200.00 350.00        6.00   

                       importe               iva_comision_db            \
status_code    0   All      -5      0    All              -5    0  All   
0           0.00  6.00  150.00   0.00 150.00            0.96 0.00 0.96   
1           0.00  0.00    0.00 100.00 100.00            0.00 0.00 0.00   
2           4.00  4.00    0.00 100.00 100.00            0.00 0.64 0.64   
3           4.00 10.00  150.00 200.00 350.00            0.96 0.64 1.60   

            serti_transaction_id          transactionId           
status_code                   -5    0 All            -5    0 All  
0                           2.00 0.00   2          2.00 0.00   2  
1                           0.00 1.00   1          0.00 1.00   1  
2                           0.00 1.00   1          0.00 1.00   1  
3                           2.00 2.00   4          2.00 2.00   4

In [51]:
pivot_carrier.shape

(4, 20)

### PIVOT RESUMEN DE COMISIONES

In [52]:
pivot_carrier_resumen = pd.pivot_table(cruce_1_true,index=['carrier_x','serti_date']
                            ,columns=['status_code']
                            ,values=['importe','amount','transactionId','serti_transaction_id','comision_db',
                                     'iva_comision_db']
                                       ,fill_value=0
                            ,aggfunc={'importe':np.sum,
                                      'amount':np.sum,
                                      'comision_db':np.sum,
                                      'iva_comision_db':np.sum,
                                      'transactionId':len,
                                      'serti_transaction_id':len})
pivot_carrier_resumen.head()

amount      comision_db    importe      iva_comision_db  \
status_code              -5    0          -5  0      -5    0              -5   
carrier_x serti_date                                                           
AT&T      2022-01-11      0  100           0  0       0  100            0.00   
TELCEL    2021-12-22    150    0           6  0     150    0            0.96   
          2022-01-11      0  100           0  4       0  100            0.00   

                          serti_transaction_id    transactionId     
status_code             0                   -5  0            -5  0  
carrier_x serti_date                                                
AT&T      2022-01-11 0.00                    0  1             0  1  
TELCEL    2021-12-22 0.00                    2  0             2  0  
          2022-01-11 0.64                    0  1             0  1

In [53]:
pivot_carrier_resumen.shape

(3, 12)

# EXCEL FINAL

In [54]:
writer = pd.ExcelWriter('CONCILIACION TOPUPS REVIEW 2022.01.17.xlsx', engine='openpyxl', options={'encoding':'utf-8'})
pivot_serti.to_excel(writer, sheet_name='RESUMEN SERTI')
pivot_true.to_excel(writer, sheet_name='RESUMEN')
pivot_carrier.to_excel(writer, sheet_name='RESUMEN CARRIER')
pivot_carrier_resumen.to_excel(writer, sheet_name='RESUMEN COMISIONES')
cruce_1_true.to_excel(writer, sheet_name='DETALLE COMPLETO',index=False)
cruce_1_final.to_excel(writer, sheet_name='FOUND',index=False)
na_cruce_1_final.to_excel(writer, sheet_name='NOT FOUND',index=False)
duplicates_cruce_1_true.to_excel(writer, sheet_name='DUPLICADOS_INDEX_TOTAL',index=False)
writer.save()
writer.close()